# Locality Data Preprocessing

- Prepares locality data (locality = ZIP code based mapping in Switzerland)
- add altitude of city centers

In [1]:
import pandas as pd
import numpy as np

import geopandas as gp
gp.options.io_engine = "pyogrio"

import pyproj
from shapely.ops import transform

import json

In [2]:
import pandas as pd
from pathlib import Path
import geopandas as gp
import re
gp.options.io_engine = "pyogrio"
from shapely.geometry import Point, Polygon
from datetime import datetime
import plotly.express as px
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape as geoshape
from shapely.geometry import Polygon
import numpy as np
from joblib import Parallel, delayed
from scipy.spatial import cKDTree
import netCDF4
import xarray as xr
import pyproj
from shapely.ops import transform

In [3]:
ws_dir = Path("/mnt/wks3/aschneuwl/workspace")

# ZIP Code - BFS Mapping
Each zip code is assigned to a BFS (politische Gemeinde)

In [3]:
zip_municipality_mapping = pd.read_csv("/home/aschneuwl/workspace/data/geo/AMTOVZ_CSV_LV95/AMTOVZ_CSV_LV95.csv", sep=";", names=["city", "plz", "addDigit", "municipality", "bfs", "kt", "e", "n", "lang", "val"])
zip_municipality_mapping = zip_municipality_mapping.tail(-1)
zip_municipality_mapping.plz = zip_municipality_mapping.plz.apply(lambda x: int(x))
zip_municipality_mapping.bfs = zip_municipality_mapping.bfs.apply(lambda x: int(x))

In [7]:
zip_municipality_mapping[zip_municipality_mapping["plz"] == 8833]

,city,plz,addDigit,municipality,bfs,kt,e,n,lang,val
174,Samstagern,8833,00,Richterswil,138,ZH,2694030.267,1227725.555,de,2008-07-01
1637,Samstagern,8833,00,Wollerau,1323,SZ,2694776.196,1227200.958,de,2008-07-01


In [4]:
zip_city = zip_municipality_mapping.groupby("plz").agg({"city": set}).reset_index()
zip_city["city"] = zip_city.city.apply(lambda x: ", ".join(sorted(list(x))))
zip_city["plz"] = zip_city.plz.astype(str)

In [24]:
kt = zip_municipality_mapping[["plz", "kt"]].dropna().drop_duplicates().drop_duplicates(subset=["plz"])
kt["plz"] = kt.plz.astype(str)

In [221]:
bfs_per_zip = zip_municipality_mapping.groupby("plz").aggregate({"bfs": "nunique"}).reset_index()
bfs_per_zip[bfs_per_zip.bfs > 1].sort_values("plz")

,plz,bfs
6,1008,3
11,1015,3
15,1023,2
18,1026,2
23,1031,2
...,...,...
3182,9630,3
3184,9633,4
3185,9642,2
3186,9643,2


In [220]:
zip_per_bfs = zip_municipality_mapping.groupby("bfs").aggregate({"plz": "nunique"}).reset_index()
zip_per_bfs[zip_per_bfs.plz > 1]

,bfs,plz
1,2,2
3,4,4
9,10,2
12,13,2
19,27,2
...,...,...
2132,7001,4
2136,7005,2
2138,7007,4
2139,7008,2


# Municipalities (Politische Gemeinden)

In [202]:
municipalities = gp.read_file("/home/aschneuwl/workspace/data/geo/swissboundaries3d_2024-01_2056_5728/swissBOUNDARIES3D_1_5_TLM_HOHEITSGEBIET.shp")

/home/aschneuwl/.pyenv/versions/3.12.2/lib/python3.12/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)
/home/aschneuwl/.pyenv/versions/3.12.2/lib/python3.12/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)


In [203]:
lv95 = pyproj.CRS('EPSG:2056')
wgs84 = pyproj.CRS('EPSG:4326')
#municipalities.geometry.simplify(0.5)
project = pyproj.Transformer.from_crs(lv95, wgs84, always_xy=True).transform

for i, shape in municipalities.iterrows():
    if shape.geometry is not None:
        shape.geometry = transform(project, shape.geometry)
        municipalities.iloc[i] = shape

In [204]:
municipalities.to_file('swissBOUNDARIES3D_1_5_TLM_HOHEITSGEBIET.shp')

/home/aschneuwl/.pyenv/versions/3.12.2/lib/python3.12/site-packages/pyogrio/raw.py:530: RuntimeWarning: Field DATUM_AEND create as date field, though DateTime requested.
  ogr_write(
/home/aschneuwl/.pyenv/versions/3.12.2/lib/python3.12/site-packages/pyogrio/raw.py:530: RuntimeWarning: Field DATUM_ERST create as date field, though DateTime requested.
  ogr_write(


## Mapping ZIP Code to the largest Municipality (BFS)
A zip code may touch the area of multile municipalities. We assign each zip code to the largest municipality.

In [206]:
bfs_surface_mapping = {municipalities.BFS_NUMMER.values[i]: municipalities.GEM_FLAECH.values[i] for i in range(len(municipalities.BFS_NUMMER.values))}

zip_largest_bfs = {}
for zip_code in list(zip_municipality_mapping.plz.unique()):
    bfs_candidates = zip_municipality_mapping[zip_municipality_mapping.plz == zip_code].bfs.values

    area_sizes = []
    for bfs in bfs_candidates:
        area_sizes.append(bfs_surface_mapping[bfs])
    
    zip_largest_bfs[int(zip_code)] = int(bfs_candidates[np.argmax(area_sizes)])

with open("zip_largest_bfs_2024.json", "w") as fp:
    json.dump(zip_largest_bfs, fp)

# Localities
Each locality has a ZIP code (a ZIP code can be assigned to multiple localities)

In [25]:
localities_zip = gp.read_file("/home/aschneuwl/workspace/data/geo/AMTOVZ_SHP_LV95/AMTOVZ_ZIP.shp")
localities_zip = localities_zip.dissolve(by='ZIP4').reset_index()
localities_zip["location_center"] = localities_zip.geometry.centroid

/home/aschneuwl/.pyenv/versions/3.12.3/lib/python3.12/site-packages/pyogrio/geopandas.py:49: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  res = pd.to_datetime(ser, **datetime_kwargs)


In [26]:
localities_zip

,ZIP4,geometry,FK_LOCALIT,ZIP_ID,ADDITIONAL,STATUS,INMODIFICA,MODIFIED,SHAPE_AREA,SHAPE_LEN,location_center
0,1000,"POLYGON ((2541155.925 1153855.760, 2541156.273...",4FDEB810-07ED-4C4B-8EA2-3F58A2C14168,126,26,REAL,FALSE,2017-02-23,3041061.8341955,9283.222431,POINT (2542854.151 1157340.310)
1,1003,"POLYGON ((2538573.608 1152334.254, 2538573.645...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,150,00,REAL,FALSE,2023-04-24,807970.256945,6827.813259,POINT (2537987.025 1152401.435)
2,1004,"POLYGON ((2536683.022 1154823.758, 2536683.662...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,151,00,REAL,FALSE,2020-10-27,2263066.100783,9257.391218,POINT (2537024.646 1153398.010)
3,1005,"POLYGON ((2539166.460 1153824.989, 2539142.547...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,152,00,REAL,FALSE,2023-04-24,1582461.6470105,11455.107855,POINT (2538961.999 1152580.198)
4,1006,"POLYGON ((2538851.320 1150757.759, 2538848.636...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,153,00,REAL,FALSE,2020-04-23,6727021.4768335,18607.733902,POINT (2538290.838 1149191.127)
...,...,...,...,...,...,...,...,...,...,...,...
3189,9652,"POLYGON ((2733053.494 1235631.236, 2733054.886...",2DDBB392-815A-4296-AC77-2EE4D18E6996,5481,00,REAL,FALSE,2022-09-23,3344529.933364,10853.545174,POINT (2732601.043 1233820.839)
3190,9655,"POLYGON ((2734132.840 1229168.036, 2734132.981...",00A7F1AE-6F66-439E-AFBE-902FD1A95D52,5478,00,REAL,FALSE,2020-12-17,12247777.917112,20434.224957,POINT (2735704.169 1229414.124)
3191,9656,"POLYGON ((2740581.285 1228650.698, 2740830.849...",04B65B23-01B5-497B-AAB3-74E380D79F70,5482,00,REAL,FALSE,2023-08-24,33266214.6835855,31682.903183,POINT (2738419.336 1226765.148)
3192,9657,"POLYGON ((2742421.325 1229886.419, 2742337.988...",C416D444-D889-4314-87C7-C2289B89DE36,5483,00,REAL,FALSE,2023-08-24,19729870.3447145,31522.228610,POINT (2741532.270 1228956.219)


In [27]:
localities_zip["longitude_x"] = localities_zip.location_center.apply(lambda p: p.x)
localities_zip["latitude_y"] = localities_zip.location_center.apply(lambda p: p.y)

In [28]:
lv95 = pyproj.CRS('EPSG:2056')
wgs84 = pyproj.CRS('EPSG:4326')
#municipalities.geometry.simplify(0.5)
project = pyproj.Transformer.from_crs(lv95, wgs84, always_xy=True).transform

for i, shape in localities_zip.iterrows():
    if shape.geometry is not None:
        shape.geometry = transform(project, shape.geometry)
        shape.location_center = transform(project, shape.location_center)
        localities_zip.iloc[i] = shape

In [29]:
localities_zip = localities_zip.set_crs(wgs84, allow_override=True)

## Altitude

In [30]:
def assign_altitude_to_locality_center(altitude_map_fpath: Path,
                                       locality_gdf: gp.GeoDataFrame,
                                       src_crs: str = "EPSG:21781",
                                       target_crs: str = "EPSG:4326") -> gp.GeoDataFrame:
    
    with rasterio.open(altitude_map_fpath) as src:
        raster_data = src.read(1)
        mask = raster_data != src.nodata
    
        shapes_generator = list(shapes(raster_data, mask=mask, transform=src.transform))
        
        def process_shape(geom, value):
            geom = rasterio.warp.transform_geom(src_crs, target_crs, geom, precision=6)
            
            return geoshape(geom), value
            
        output = Parallel(n_jobs=-1)(delayed(process_shape)(geom, value) for geom, value in shapes_generator)
        polygons, values = zip(*output)
    
    gdf_polygons = gp.GeoDataFrame({'altitude': values, 'geometry': polygons}, crs=target_crs)
    gdf_polygons["center"] = None
    
    lv95 = pyproj.CRS('EPSG:2056')
    wgs84 = pyproj.CRS('EPSG:4326')
    project_to_lv95 = pyproj.Transformer.from_crs(wgs84, lv95, always_xy=True).transform
    project_to_wgs84 = pyproj.Transformer.from_crs(lv95, wgs84, always_xy=True).transform

    def center_helper(shape):
        center = None
        if shape.geometry is not None:
            center = transform(project_to_wgs84, transform(project_to_lv95, shape.geometry).centroid)

        return center

    centers = Parallel(n_jobs=-1)(delayed(center_helper)(shape) for _, shape in gdf_polygons.iterrows())
    gdf_polygons["center"] = centers
    
    # Assign each locality center to the corresponding altitude
    height_search_tree = cKDTree(gdf_polygons.center.apply(lambda geom: (geom.x, geom.y)).tolist())
    distances, indices = height_search_tree.query(locality_gdf.location_center.apply(lambda geom: (geom.x, geom.y)).tolist())
    locality_gdf["altitude"] = gdf_polygons.iloc[indices].altitude.values
    
    # Factor 100 for station matching
    locality_gdf["altitude_100"] = gdf_polygons.iloc[indices].altitude.values * 100

    return locality_gdf

In [31]:
altitude_map_fpath = Path("/home/aschneuwl/workspace/data/geo/height/DHM200.asc")
localities_zip = assign_altitude_to_locality_center(altitude_map_fpath, localities_zip)

In [32]:
localities_zip

,ZIP4,geometry,FK_LOCALIT,ZIP_ID,ADDITIONAL,STATUS,INMODIFICA,MODIFIED,SHAPE_AREA,SHAPE_LEN,location_center,longitude_x,latitude_y,altitude,altitude_100
0,1000,"POLYGON ((6.67161 46.53340, 6.67162 46.53344, ...",4FDEB810-07ED-4C4B-8EA2-3F58A2C14168,126,26,REAL,FALSE,2017-02-23,3041061.8341955,9283.222431,POINT (6.693 46.565),2.542854e+06,1.157340e+06,863.598999,86359.899902
1,1003,"POLYGON ((6.63816 46.51948, 6.63816 46.51948, ...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,150,00,REAL,FALSE,2023-04-24,807970.256945,6827.813259,POINT (6.631 46.520),2.537987e+06,1.152401e+06,479.898987,47989.898682
2,1004,"POLYGON ((6.61318 46.54170, 6.61319 46.54164, ...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,151,00,REAL,FALSE,2020-10-27,2263066.100783,9257.391218,POINT (6.618 46.529),2.537025e+06,1.153398e+06,515.500000,51550.000000
3,1005,"POLYGON ((6.64569 46.53295, 6.64539 46.53194, ...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,152,00,REAL,FALSE,2023-04-24,1582461.6470105,11455.107855,POINT (6.643 46.522),2.538962e+06,1.152580e+06,548.901978,54890.197754
4,1006,"POLYGON ((6.64198 46.50533, 6.64196 46.50480, ...",7FBCC48A-F9B6-4BCD-95CA-A3A827209D84,153,00,REAL,FALSE,2020-04-23,6727021.4768335,18607.733902,POINT (6.635 46.491),2.538291e+06,1.149191e+06,372.902008,37290.200806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,9652,"POLYGON ((9.19664 47.25818, 9.19665 47.25794, ...",2DDBB392-815A-4296-AC77-2EE4D18E6996,5481,00,REAL,FALSE,2022-09-23,3344529.933364,10853.545174,POINT (9.190 47.242),2.732601e+06,1.233821e+06,927.994019,92799.401855
3190,9655,"POLYGON ((9.20897 47.19984, 9.20897 47.19984, ...",00A7F1AE-6F66-439E-AFBE-902FD1A95D52,5478,00,REAL,FALSE,2020-12-17,12247777.917112,20434.224957,POINT (9.230 47.202),2.735704e+06,1.229414e+06,929.908997,92990.899658
3191,9656,"POLYGON ((9.29388 47.19385, 9.29711 47.19183, ...",04B65B23-01B5-497B-AAB3-74E380D79F70,5482,00,REAL,FALSE,2023-08-24,33266214.6835855,31682.903183,POINT (9.265 47.177),2.738419e+06,1.226765e+06,1100.201050,110020.104980
3192,9657,"POLYGON ((9.31855 47.20457, 9.31743 47.20406, ...",C416D444-D889-4314-87C7-C2289B89DE36,5483,00,REAL,FALSE,2023-08-24,19729870.3447145,31522.228610,POINT (9.307 47.196),2.741532e+06,1.228956e+06,906.991028,90699.102783


In [33]:
localities_zip = localities_zip.merge(kt, left_on="ZIP4", right_on="plz").drop("plz", axis=1)
localities_zip = localities_zip.merge(zip_city, left_on="ZIP4", right_on="plz").drop("plz", axis=1)

In [34]:
ws2 = Path("/mnt/wks3/aschneuwl/workspace/data/preprocessed")

In [35]:
localities_zip.to_parquet(ws2 / Path("swiss_localities_with_altitudes.parquet"))